In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.iolib.summary2 import summary_col
!pip install linearmodels==4.24
from linearmodels import PanelOLS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 28.9 MB/s 


In [ ]:

n=2000
df1=pd.DataFrame(index=range(n))
df1['ability']=np.random.normal(100,15,n)
df1['year']=1
df1['persnr']=df1.index
df1['training']=0
## Now copy the DataFrame (i.e. generate observations for second year)
df2=df1.copy()
df2['year']=2
## Training only in year 2:
df2['training']=(df2.ability+np.random.normal(0,10,n)>=100)
## Generate DataFrame that spans both years by appending the two data frames
df=pd.concat([df1,df2], sort=False)
df['sales']= 10000 + df.training*5000 + df.ability*100 + df.year*2000    + np.random.normal(0,4000,2*n)

df

,ability,year,persnr,training,sales
0,107.506795,1,0,0,19840.014799
1,100.083488,1,1,0,23172.343561
2,102.208539,1,2,0,16434.416462
3,117.644839,1,3,0,18844.026536
4,69.337476,1,4,0,13179.632451
...,...,...,...,...,...
1995,95.097568,2,1995,0,31595.775530
1996,83.305100,2,1996,0,19547.685395
1997,102.512326,2,1997,1,30744.229036
1998,83.071159,2,1998,1,24343.380858


In [ ]:
reg1=smf.ols('sales ~ training + C(year)', data=df).fit()
reg2=smf.ols('sales ~ training + C(year) + C(persnr)', data=df).fit()

print(summary_col([reg1,reg2]))


                   sales I     sales II 
----------------------------------------
C(persnr)[T.1000]            -562.7354  
                             (4037.5554)
C(persnr)[T.1001]            1248.7202  
                             (4035.5382)
C(persnr)[T.1002]            -3302.8958 
                             (4035.5382)
C(persnr)[T.1003]            -5104.9121 
                             (4037.5554)
C(persnr)[T.1004]            374.4752   
                             (4035.5382)
C(persnr)[T.1005]            -5699.4703 
                             (4035.5382)
C(persnr)[T.1006]            -596.4493  
                             (4035.5382)
C(persnr)[T.1007]            1432.4269  
                             (4035.5382)
C(persnr)[T.1008]            -6805.6315 
                             (4037.5554)
C(persnr)[T.1009]            -4924.7358 
                             (4035.5382)
C(persnr)[T.100]             -1949.5793 
                             (4035.5382)
C(persnr)[T.101

In [ ]:
df=df.set_index(['persnr','year'])
regP=PanelOLS.from_formula('sales ~ training + EntityEffects + TimeEffects', data=df).fit()
print(regP)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  sales   R-squared:                        0.1515
Estimator:                   PanelOLS   R-squared (Between):              0.1008
No. Observations:                4000   R-squared (Within):               0.3686
Date:                Mon, Nov 14 2022   R-squared (Overall):              0.1071
Time:                        07:53:33   Log-likelihood                 -3.75e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      356.65
Entities:                        2000   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                  F(1,1998)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             356.65
                            